In [27]:
import abc
from pydantic import BaseModel, Extra
from typing import Any, List, Optional, TypeVar, Union

class MyBaseModel(BaseModel):

    class Config:
        allow_population_by_field_name = True
        extra = Extra.allow

class StratTypeMappingItem(BaseModel):
    type: str
    name: str
    value: str
    bin_count: Optional[int] = None
    top_count: Optional[int] = None
    max_unique: Optional[int] = None

class Server(MyBaseModel): 
    name: str
    database: str
    dialect: str
    driver: Optional[str] = None
    host: Optional[str] = ""
    password: Optional[str] = None
    user: Optional[str] = None
    port: Optional[str] = None
    schemas: Optional[list[str]] = []

class DataMap(MyBaseModel):
    pass
    
class Computation(MyBaseModel, abc.ABC):
    name: str
    value: str
    columns: Optional[List[str]] = []

class TopCounts(Computation):
    top_count: Optional[int] = None
    max_unique: Optional[int] = None
    
class Histogram(Computation):
    binning_algo: Optional[str] = ""
    bin_count: Optional[int] = None
    
     
class Configuration(MyBaseModel):
    run_type: Optional[str] = ""
    servers: Optional[List[Server]] = []
    datatype_mapping: Optional[List[DataMap]]
    default_partition_key: Optional[str] = None
    computations: Optional[List[Union[Histogram, TopCounts]]] = []
    

class StratifyConfig(Configuration):
     strat_type_mapping: List[StratTypeMappingItem]
     servers: list[Server]
     src_server: str 

class Context(MyBaseModel):
    config: Configuration
    job_id: int
    runtime_params: Optional[dict] = {}

    def __getattr__(self, name):
        #print("checking attributes")
        if hasattr(self.config, name):
            value = getattr(self.config, name)
            return value
        else:
            return None
        
        
def get_from_list(lyst: str, name: str) -> BaseModel: 
    for item in lyst:
        if item.name == name:
            return item
    return None

In [39]:
import pandas as pd 
compute_obj = {'name': 'histogram', 'value': '$1', 'binning_algo': 
               'clean_many', 'bin_count': 10}

conf = Configuration()
c = Context(config=conf, job_id=-9)

hel = getattr(c, "hello")
print(hel)
setattr(c, "hello", "there")
hel = getattr(c, "hello")
print(hel)

None
there


In [44]:
import pandas as pd
avg = 2342
count = 76

avg_count = pd.DataFrame(index=["average", "count"], data=[avg,count])
avg_count2 = pd.DataFrame(index=["average", "count"], data=[avg+346,count+232])

df = pd.concat([avg_count, avg_count2], axis=1)
df

,0,0
average,2342,2688
count,76,308


In [ ]:
# Rules for Context usage
# 1. Only set attributes via a function to insure that all are set as properties and not dicts
# 2. Use a get_or_create function to eliminate -> x = getattr(); if not x: create x;